In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
############################################################
# pyupbit.get_current_price("KRW-BTC") : BIT 현재가
# --> price_KRW_BTC : BTC 현재가 
# upbit.get_balance(ticker="KRW-BTC") : BIT 잔고 조회 (My_Wallet_BIT)
# My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
# My_Wallet_Money = upbit.get_balance(ticker="KRW")
# My_Wallet_BITWON = float(price_KRW_BTC * My_Wallet_BIT))
# 
# print(upbit.get_balance("KRW-XRP"))     # KRW-XRP 조회
# print(upbit.get_balance("KRW"))         # 보유 현금 조회
# 
############################################################
import os
import jwt
import uuid
import hashlib
import pyupbit
import datetime, time# for sleep
from urllib.parse import urlencode

import requests

access_key = os.environ['UPBIT_OPEN_API_ACCESS_KEY']
secret_key = os.environ['UPBIT_OPEN_API_SECRET_KEY']
server_url = os.environ['UPBIT_OPEN_API_SERVER_URL']
upbit = pyupbit.Upbit(access_key, secret_key)

payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
}

jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
authorize_token = 'Bearer {}'.format(jwt_token)
headers = {"Authorization": authorize_token}

res = requests.get(server_url + "/v1/accounts", headers=headers)

# print(res.json())
# [{'currency': 'BTC', 'balance': '0.00026', 'locked': '0.0', 'avg_buy_price': '54041000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}]

############################################################
# BTC 조건
Daily_Check_BTC = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=365)
Daily_Check_BTC['diff1'] = round(((Daily_Check_BTC['high'] - Daily_Check_BTC['low']) / Daily_Check_BTC['low']) * 100,2)
Daily_Check_BTC['diff2'] = round(((Daily_Check_BTC['open'] - Daily_Check_BTC['close']) / Daily_Check_BTC['open']) * 100,2)
Daily_Check_BTC_Minus = Daily_Check_BTC[Daily_Check_BTC['diff2'] < 0]
print("# BTC 마이너스만..")
print(Daily_Check_BTC_Minus['diff2'].max())
print(Daily_Check_BTC_Minus['diff2'].min())
print(Daily_Check_BTC_Minus['diff2'].mean() * 2) # 엥???? 
BTC_Lower_Value_V = round(Daily_Check_BTC_Minus['diff2'].mean() * 2,1)
############################################################
# ETH 조건
Daily_Check_ETH = pyupbit.get_ohlcv("KRW-ETH", interval="day", count=365)
Daily_Check_ETH['diff1'] = round(((Daily_Check_ETH['high'] - Daily_Check_ETH['low']) / Daily_Check_ETH['low']) * 100,2)
Daily_Check_ETH['diff2'] = round(((Daily_Check_ETH['open'] - Daily_Check_ETH['close']) / Daily_Check_ETH['open']) * 100,2)
Daily_Check_ETH_Minus = Daily_Check_ETH[Daily_Check_ETH['diff2'] < 0]
print("# ETH 마이너스만..")
print(Daily_Check_ETH_Minus['diff2'].max())
print(Daily_Check_ETH_Minus['diff2'].min())
print(Daily_Check_ETH_Minus['diff2'].mean() * 2) # 엥???? 
ETH_Lower_Value_V = round(Daily_Check_ETH_Minus['diff2'].mean() * 2,1)
############################################################
BTC_Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
BTC_Lower_Value = float(-1.5)
# BTC_Lower_Value = float(BTC_Lower_Value_V)

ETH_Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
ETH_Lower_Value = float(-1.5)
# ETH_Lower_Value = float(ETH_Lower_Value_V)
############################################################

def Current_BTC() :
    check_get_type = 0
    while check_get_type == 0 :
        time.sleep(1)
        TEMP_BTC_Current = pyupbit.get_current_price("KRW-BTC") # 현재 BIT가격
        if isinstance(TEMP_BTC_Current, float) :
            check_get_type = 1
            print("# WHILE - TEMP_BTC_Current : %s" % TEMP_BTC_Current)
            return TEMP_BTC_Current

def Current_ETH() :
    check_get_type = 0
    while check_get_type == 0 :
        time.sleep(1)
        TEMP_ETH_Current = pyupbit.get_current_price("KRW-ETH") # 현재 BIT가격
        if isinstance(TEMP_ETH_Current, float) :
            check_get_type = 1
            print("# WHILE - TEMP_ETH_Current : %s" % TEMP_ETH_Current)
            return TEMP_ETH_Current

# price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # BIT 현재가
# price_KRW_ETH = pyupbit.get_current_price("KRW-ETH") # BIT 현재가
price_KRW_BTC = Current_BTC()
price_KRW_ETH = Current_ETH()
Min_Call_Price = float(10000)
Min_Call_BTC_Volume = 10000 / price_KRW_BTC # 10000원을 현대 BIT가로 나눈값
Min_Call_ETH_Volume = 10000 / price_KRW_ETH # 10000원을 현대 BIT가로 나눈값

def Call_Condition(File_Name) :

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
    }

    jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)

    # 자산 종류 수로 내가 가진 자산을 확인 2이면 BIT와 KRW, 1개면 KRW로 판단 (BIT없음)
    # ---> 1개일 경우 종류를 확인해봐야할 듯... (2021.03.30)

    My_Wallet_KRW, My_Wallet_BTC, My_Wallet_ETH = {}, {}, {}
    if len(res.json()) >= 2 :
        for x in res.json() :
            if x["currency"] == "KRW" :
                My_Wallet_KRW = x
            elif x["currency"] == "BTC" :
                My_Wallet_BTC = x
            elif x["currency"] == "ETH" :
                My_Wallet_ETH = x
            else :
                print("# ELSE")
        if len(My_Wallet_BTC) == 0 :
            __Make_Put("BUY", "BTC", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소BIT, File_Name
        if len(My_Wallet_ETH) == 0 :
            __Make_Put("BUY", "ETH", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소ETH, File_Name
    else :
        print("# res.json LENGTH is Wrong")

    ############################################################
    # Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
    # Lower_Value = float(-15) # 물타기라인, UPBit 거래수수료
    # Min_Call_Price = float(10000)
    # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # BIT 현재가
    # Min_Call_BTC_Volume = 10000 / price_KRW_BTC
    print("# 최소매수가능 금액 : %f" % Min_Call_Price)
    print("# 최소매수가능 BIT : %f" % Min_Call_BTC_Volume)
    print("# 최소매수가능 ETH : %f" % Min_Call_ETH_Volume)
    ############################################################

    ############################################################
    Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10
    # Call_Price_Possible = float(price_KRW_BTC * My_Wallet_BIT) / 10
    Call_Volume_Possible = upbit.get_balance(ticker="KRW-BTC") / 10 # 보유 BIT의 1/10
    print("# 1/10 Volume = %f" % Call_Volume_Possible)
    output.write("# 1/10 Volume = %f\n" % Call_Volume_Possible)
    print("# 1/10 Price = %f" % Call_Price_Possible)
    output.write("# 1/10 Price = %f\n" % Call_Price_Possible)
    print("")
    output.write("\n")
    ############################################################

    if Call_Price_Possible < Min_Call_Price : # 가진 자산의 1/10을 매도하거나 매수하지만 기본금액에 모자라면 기본금액으로 설정
        i = 1
        Temp_Possible = 0
        print("Min_Call_Price : %s" % Min_Call_Price)
        print("Min_Call_Price : %s" % type(Min_Call_Price)) # <class 'float'>
        print("Call_Price_Possbile : %s" % Call_Price_Possible)
        print("Call_Price_Possbile : %s" % type(Call_Price_Possible)) # <class 'float'>
        while Call_Price_Possible < Min_Call_Price : # 현재가 변동으로 10%를 추가로 잡아줌
            if Temp_Possible == 0 : 
                Temp_Possible = Call_Price_Possible
            Call_Price_Possible = Temp_Possible * i
            print("# %s번째 Price : %s" % (i, Call_Price_Possible))
            output.write("# %s번째 Price : %s\n" % (i, Call_Price_Possible))
            i += 1
            if i > 10 :
                Call_Price_Possible = Min_Call_Price

        print("# 완료 %s번째 Price : %s" % (i, Call_Price_Possible))
        output.write("# 완료 %s번째 Price : %s\n" % (i, Call_Price_Possible))
        print("")
        output.write("\n")

    # Temp_Volume = Call_Volume_Possible * price_KRW_BTC
    if Call_Volume_Possible < Min_Call_BTC_Volume : # 가진 자산의 1/10을 매도하거나 매수하지만 기본금액에 모자라면 기본금액으로 설정
        i = 1
        while (Call_Volume_Possible * price_KRW_BTC) < Min_Call_Price :
            print("# %s번째 Volume : %f" % (i, Call_Volume_Possible))
            output.write("# %s번째 Volume : %f\n" % (i, Call_Volume_Possible))
            Call_Volume_Possible = Call_Volume_Possible * i
            # if i < 0 :
            #     Call_Volume_Possible = price_KRW_BTC / Min_Call_Price
            #     break
            i += 1
        print("# 완료 %s번째 Volume : %f" % (i, Call_Volume_Possible))
        output.write("# 완료 %s번째 Volume : %f\n" % (i, Call_Volume_Possible))
        print("")
        output.write("\n")

    print("# 최종가격대")
    output.write("# 최종가격대\n")
    print("# Call_Price_Possible = %f" % Call_Price_Possible)
    output.write("# Call_Price_Possible = %f\n" % Call_Price_Possible)
    print("# Call_Volume_Possible = %f" % Call_Volume_Possible)
    output.write("# Call_Volume_Possible = %f\n" % Call_Volume_Possible)
    print("")
    output.write("\n")

    output.close()

    return Call_Price_Possible, Call_Volume_Possible

def __Make_OrderLists(COIN) :

    Total_Order = []
    Trade_List_uuid, Trade_List_side, Trade_List_date = [], [], []
    if COIN == 'KRW-BTC' :
        # print("# KRW-BTC 주문리스트 조회")
        for Dist in ('done','cancel') :
            query = {
            'market': 'KRW-BTC',
            'state': Dist, # 주문 상태 , wait : 체결 대기 (default),  done : 전체 체결 완료,  cancel : 주문 취소
            }
            query_string = urlencode(query).encode()
            m = hashlib.sha512()
            m.update(query_string)
            query_hash = m.hexdigest()
            payload = {
                'access_key': access_key,
                'nonce': str(uuid.uuid4()),
                'query_hash': query_hash,
                'query_hash_alg': 'SHA512',
            }
            jwt_token = jwt.encode(payload, secret_key)
            authorize_token = 'Bearer {}'.format(jwt_token)
            headers = {"Authorization": authorize_token}
            res = requests.get(server_url + "/v1/orders", params=query, headers=headers)
            for res_list in range(len(res.json())) :
                Trade_List_uuid.append(res.json()[res_list]["uuid"])
                Trade_List_side.append(res.json()[res_list]["side"])
                Trade_List_date.append(res.json()[res_list]["created_at"])
    elif COIN == 'KRW-ETH' :
        # print("# KRW-ETH 주문리스트 조회")
        for Dist in ('done','cancel') :
            query = {
            'market': 'KRW-ETH',
            'state': Dist, # 주문 상태 , wait : 체결 대기 (default),  done : 전체 체결 완료,  cancel : 주문 취소
            }
            query_string = urlencode(query).encode()
            m = hashlib.sha512()
            m.update(query_string)
            query_hash = m.hexdigest()
            payload = {
                'access_key': access_key,
                'nonce': str(uuid.uuid4()),
                'query_hash': query_hash,
                'query_hash_alg': 'SHA512',
            }
            jwt_token = jwt.encode(payload, secret_key)
            authorize_token = 'Bearer {}'.format(jwt_token)
            headers = {"Authorization": authorize_token}
            res = requests.get(server_url + "/v1/orders", params=query, headers=headers)
            for res_list in range(len(res.json())) :
                Trade_List_uuid.append(res.json()[res_list]["uuid"])
                Trade_List_side.append(res.json()[res_list]["side"])
                Trade_List_date.append(res.json()[res_list]["created_at"])
    else : # bid : 매수, ask : 매도
        print("# BTC/ETH 주문내역 조회 : 오류")
        # ---> 매수는 cancel 로 잡히고 매도는 done으로 잡힌다
        # ---> 매수 시 시장가로 매수하니 소수점이하..어쩌고 잔량이 어쩌고..

    Now_DATE = datetime.datetime.today().strftime("%Y-%m-%d")

    # for res_dict in res.json() : # <class 'dict'>
    #     for keys, values in res_dict.items() :
    #         DATE_HIST, SIDE_HIST = 0, 0
    #         if keys == 'created_at' :
    #             # print(values[:10]) # <class 'str'> 2021-02-20T10:41:25+09:00
    #             # print(Now_DATE)
    #             if values[:10] == Now_DATE :
    #                 Trade_List_uuid.append(res_dict['uuid'])
    #                 Trade_List_side.append(res_dict['side'])
    #                 Trade_List_date.append(res_dict['created_at'])
    
    # List에 아무것도 안담기는 경우는???
    # print(Trade_List_uuid) # <class 'list'>
    # print(Trade_List_side) # <class 'list'>
    # print(Trade_List_date) # <class 'list'>
    
    Trade_List_date_10, Trade_List_side_10, idx_list = [], [], []
    for x in Trade_List_date :
        Trade_List_date_10.append(x[:10])
    # print(Trade_List_date_10)
    for idx, val in enumerate(Trade_List_date_10) :
        if val == Now_DATE :
           idx_list.append(idx) 
    # print(idx_list)
    for idx, val in enumerate(Trade_List_side) :
        if idx in idx_list :
            # print("%s : %s" % (Trade_List_date_10[idx], val))
            Trade_List_side_10.append(val)
    # print("# 길이 : %s" % len(Trade_List_uuid))
    # print("# 길이 : %s" % len(Trade_List_side))
    # print("# 길이 : %s" % len(Trade_List_date))

    print("# BID Count : %s" % Trade_List_side.count('bid')) # bid : 매수, ask : 매도
    print("# ASK Count : %s" % Trade_List_side.count('ask')) # bid : 매수, ask : 매도
    print("# Real BID Count #")
    print("# BID Count : %s" % Trade_List_side_10.count('bid')) # bid : 매수, ask : 매도

    return Trade_List_side_10.count('bid') # 매수 건수 만 Return

        # {'uuid': '418f0313-e001-4cc3-9dcc-f656dd24f3b2', uuid	주문의 고유 아이디	String
        #  'side': 'ask', side	주문 종류	String
        #  'ord_type': 'market', ord_type	주문 방식	String
        #  'price': None, price	주문 당시 화폐 가격	NumberString
        #  'state': 'done', state	주문 상태	String
        #  'market': 'KRW-BTC', market	마켓의 유일키	String
        #  'created_at': '2021-02-20T10:40:42+09:00', created_at	주문 생성 시간	DateString
        #  'volume': '0.000078', volume	사용자가 입력한 주문 양	NumberString
        #  'remaining_volume': '0.0', remaining_volume	체결 후 남은 주문 양	NumberString
        #  'reserved_fee': '0.0', reserved_fee	수수료로 예약된 비용	NumberString
        #  'remaining_fee': '0.0', remaining_fee	남은 수수료	NumberString
        #  'paid_fee': '2.509884', paid_fee	사용된 수수료	NumberString
        #  'locked': '0.0', locked	거래에 사용중인 비용	NumberString
        #  'executed_volume': '0.000078', executed_volume	체결된 양	NumberString
        #  'trades_count': 1} trade_count	해당 주문에 걸린 체결 수	Integer

def Detail_Trade(Trade_List_uuid) :

    print("# Detail_Trace Funtion START")

    for UUID in Trade_List_uuid :
        query = {
            'uuid': UUID,
        }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
            'query_hash': query_hash,
            'query_hash_alg': 'SHA512',
        }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/order", params=query, headers=headers)

        # print(res.json())
        Detail_Trade_Value = res.json()
        print(Detail_Trade_Value['side'])

    print("# Detail_Trace Funtion DONE")

def __Make_Put(CALL, COIN, Call_Possible, Min_Call_Price, OneDay_BUY_Count, File_Name) :

    # Call_Possible 값은 <--- My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT
    # Min_Call_Price = float(10000)

    output = open(File_Name, "a")
    print("# %s : __Make_Put Function Start" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    output.write("# %s : __Make_Put Function Start\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

    print("# %s : CALL Volume/Price : %f개 or 원" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Possible))
    output.write("# %s : CALL Volume/Price : %f개 or 원\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Possible))
    # 시장가 주문은 ord_type 필드를 price or market 으로 설정해야됩니다.
    if CALL == "SELL" :
        # 매도 주문의 경우 ord_type을 market로 설정하고 price을 null 혹은 제외해야됩니다.
        # Call_Possible = upbit.get_balance(ticker="KRW-BTC") # 매도할 때는 전체를 매도하니간 굳이 계산없이 전체
        if COIN == "BTC" :
            print("# %s : BIT 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-BTC"))) # 특정 코인이나 원화의 잔고만 조회
            output.write("# %s : BIT 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-BTC"))) # 특정 코인이나 원화의 잔고만 조회
            price_KRW_BTC = Current_BTC()
            # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
            TEMP_Call_Possible = Call_Possible * price_KRW_BTC # 현재 보유 BIT개수 * 현재 BIT값
            if TEMP_Call_Possible < Min_Call_Price : # 최소금액에 못미치면 그냥 returne
                print("# %s :---> 매도 BTC가 최소금액이 안됨" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
                output.write("# %s :---> 매도 BTC가 최소금액이 안됨\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
                return None
            else :
                print("# %s : Make call SELL -> BTC" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : Make call SELL -> BTC\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                CALL = "ask" # 매도
                ORD_TYPE = "market"
                PRICE = ''
                VOLUME = Call_Possible
        elif COIN == "ETH" :
            print("# %s : ETH 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-ETH"))) # 특정 코인이나 원화의 잔고만 조회
            output.write("# %s : ETH 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-ETH"))) # 특정 코인이나 원화의 잔고만 조회
            price_KRW_BTC = Current_BTC()
            price_KRW_ETH = Current_ETH()
            # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
            # price_KRW_ETH = pyupbit.get_current_price("KRW-ETH") # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
            TEMP_Call_Possible = Call_Possible * price_KRW_ETH # 현재 보유 BIT개수 * 현재 BIT값
            if TEMP_Call_Possible < Min_Call_Price : # 최소금액에 못미치면 그냥 returne
                print("# %s :---> 매도 ETH가 최소금액이 안됨" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
                output.write("# %s :---> 매도 ETH가 최소금액이 안됨\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
                return None
            else :
                print("# %s : Make call SELL -> ETH" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : Make call SELL -> ETH\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                CALL = "ask" # 매도
                ORD_TYPE = "market"
                PRICE = ''
                VOLUME = Call_Possible

    elif CALL == "BUY" :
        OneDay_BUY_Count += 1
        if OneDay_BUY_Count > 3 :
            return "3"
        # 매수 주문의 경우 ord_type을 price로 설정하고 volume을 null 혹은 제외해야됩니다.
        print("# %s : Make call BUY" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.write("# %s : Make call BUY\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        CALL = "bid" # 매수
        ORD_TYPE = "price"
        VOLUME = ''
        PRICE = Call_Possible
    else :
        print("# %s : Make Call ERROR~!" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.write("# %s : Make Call ERROR~!\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

    if COIN == "BTC" :
        query = {
        'market': 'KRW-BTC',
        'side': CALL, # 주문 종류 (필수) - bid : 매수, ask : 매도
        'volume': VOLUME, # 주문 수량, 주문량 (지정가, 시장가 매도 시 필수) NumberString
        'price': PRICE, # 유닛당 주문 가격
        'ord_type': ORD_TYPE,
        }
    elif COIN == "ETH" :
        query = {
        'market': 'KRW-ETH',
        'side': CALL, # 주문 종류 (필수) - bid : 매수, ask : 매도
        'volume': VOLUME, # 주문 수량, 주문량 (지정가, 시장가 매도 시 필수) NumberString
        'price': PRICE, # 유닛당 주문 가격
        'ord_type': ORD_TYPE,
        }
    else :
        print("# 주문오류~~~")
        output.write("# 주문오류~~~\n")
    
    print(query)

        # 주문 타입 (필수) - limit : 지정가 주문, price : 시장가 주문(매수), market : 시장가 주문(매도)
        # identifier : 조회용 사용자 지정값 (선택) String (Uniq 값 사용)
    query_string = urlencode(query).encode()

    print("# Query : ") # {'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 6689.808521999999, 'ord_type': 'price'}
    print(query)
    print(type(query))
    output.write("# Query : ") # {'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 6689.808521999999, 'ord_type': 'price'}
    output.write(str(query))

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, secret_key) # .decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    # print(jwt_token)
    # print(authorize_token)
    # print(headers)
    res = requests.post(server_url + "/v1/orders", params=query, headers=headers)

    print("# %s : __Make_Put : Complete\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    output.write("# %s : __Make_Put : Complete\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    # print(type(res.text)) # <class 'str'>
    print(res.text)
    print("")
     # 매수 : {"error":{"message":"주문가능한 금액(BTC)이 부족합니다.","name":"insufficient_funds_ask"}}
     # 매수 : {"uuid":"9ddd3561-1188-4e5a-83fb-5810bb7f0c94","side":"bid","ord_type":"price","price":"6689.8","state":"wait","market":"KRW-BTC","created_at":"2021-02-27T09:00:29+09:00","volume":null,"remaining_volume":null,"reserved_fee":"3.3449","remaining_fee":"3.3449","paid_fee":"0.0","locked":"6693.1449","executed_volume":"0.0","trades_count":0}
    output_res_text = res.text
    output.write("# %s : %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), output_res_text))
    output.write("\n")

    time.sleep(3) # ---> 매도하고나서 바로 또 매도하려고 함. (매도했는데 매도하려는 물량이 없어서 최소수량보다 작다는 메세지 출력됨)

    output.close()

def Current_Query(OneDay_BUY_Count, File_Name) :

    print("# Current_Query Function Start")

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
    }

    jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)

    print(res.json())
    print(type(res.json()))

    My_Wallet_KRW, My_Wallet_BTC, My_Wallet_ETH = {}, {}, {}
    if len(res.json()) >= 2 :
        for x in res.json() :
            if x["currency"] == "KRW" :
                My_Wallet_KRW = x
            elif x["currency"] == "BTC" :
                My_Wallet_BTC = x
            elif x["currency"] == "ETH" :
                My_Wallet_ETH = x
            else :
                print("# ELSE")
        if len(My_Wallet_BTC) == 0 :
            __Make_Put("BUY", "BTC", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소BIT, File_Name
            time.sleep(1)
        if len(My_Wallet_ETH) == 0 :
            __Make_Put("BUY", "ETH", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소ETH, File_Name
            time.sleep(1)
    else :
        print("# res.json LENGTH is 1 이하")
        output.write("# res.json LENGTH is 1 이하")

    if len(My_Wallet_BTC) != 0 :
        print("# BTC : %s" % My_Wallet_BTC["balance"]) # <class 'str'>
    if len(My_Wallet_ETH) != 0 :
        print("# ETH : %s" % My_Wallet_ETH["balance"]) # <class 'str'>

    # if not isinstance(My_Wallet_BTC, float) :
    if isinstance(My_Wallet_ETH, dict) :
        if len(My_Wallet_BTC) == 6 :
            TEMP_BTC_My_Wallet = float(My_Wallet_BTC['avg_buy_price'])
            # My_Wallet_BTC = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
            # TEMP_BTC_Current = pyupbit.get_current_price("KRW-BTC")
            TEMP_BTC_Current = Current_BTC()
            if int(TEMP_BTC_My_Wallet) <= 0 :
                print("# TEMP My Wallet : %f ---> 보유 BTC가 없음" % TEMP_BTC_My_Wallet)
                output.write("# TEMP My Wallet : %f ---> 보유 BTC가 없음\n" % TEMP_BTC_My_Wallet)
                return TEMP_BTC_My_Wallet
            TEMP_BTC_Percent = (TEMP_BTC_Current - TEMP_BTC_My_Wallet) / TEMP_BTC_My_Wallet * 100
            TEMP_BTC_Percent = round(TEMP_BTC_Percent, 2)
            print("# 현재 수익률 : %f " % TEMP_BTC_Percent) # <class 'float'> 
        else :
            __Make_Put("BUY", "BTC", 10000, 10000, OneDay_BUY_Count, File_Name)
            time.sleep(3) # 매수하고 바로 계좌조회는 에러가 나는거 같아서 추가 ---> TypeError: must be real number, not dict

    # if not isinstance(My_Wallet_ETH, float) :
    if isinstance(My_Wallet_ETH, dict) :
        if len(My_Wallet_ETH) == 6 :
            TEMP_ETH_My_Wallet = float(My_Wallet_ETH['avg_buy_price'])
            # My_Wallet_ETH = upbit.get_balance(ticker="KRW-ETH") # <class 'float'>
            TEMP_ETH_Current = Current_ETH()
            # TEMP_ETH_Current = pyupbit.get_current_price("KRW-ETH")
            if int(TEMP_ETH_My_Wallet) <= 0 :
                print("# TEMP My Wallet : %f ---> 보유 ETH가 없음" % TEMP_ETH_My_Wallet)
                output.write("# TEMP My Wallet : %f ---> 보유 ETH가 없음\n" % TEMP_ETH_My_Wallet)
                return TEMP_ETH_My_Wallet
            TEMP_ETH_Percent = (TEMP_ETH_Current - TEMP_ETH_My_Wallet) / TEMP_ETH_My_Wallet * 100
            TEMP_ETH_Percent = round(TEMP_ETH_Percent, 2)
            print("# 현재 수익률 : %f " % TEMP_ETH_Percent) # <class 'float'> 
        else :
            __Make_Put("BUY", "ETH", 10000, 10000, OneDay_BUY_Count, File_Name)
            time.sleep(3) # 매수하고 바로 계좌조회는 에러가 나는거 같아서 추가 ---> TypeError: must be real number, not dict

    print("############################################################")
    output.write("############################################################\n")

    if len(My_Wallet_BTC) != 0 :
        print("# BTC 현재가 : %s 원" % price_KRW_BTC)
        output.write("# KRW-BTC 현재가 : %s 원\n" % price_KRW_BTC)
        print("# BIT 잔고 조회 : %s" % My_Wallet_BTC["balance"]) # 특정 코인이나 원화의 잔고만 조회
        output.write("# KRW-BTC 현재가 : %s 원\n" % price_KRW_BTC)
        # My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
        BIT_WON = float(My_Wallet_BTC["balance"]) * price_KRW_BTC
        print("# 현재 BIT원화 잔고 : %f 원, 수익율 : %f" % (BIT_WON, TEMP_BTC_Percent))
        output.write("# 현재 BIT원화 잔고 : %f 원, 수익율 : %f" % (BIT_WON, TEMP_BTC_Percent))
    
    if len(My_Wallet_ETH) != 0 :
        print("# ETH 현재가 : %s 원" % price_KRW_ETH)
        output.write("# KRW-ETH 현재가 : %s 원\n" % price_KRW_ETH)
        print("# ETH 잔고 조회 : %s" % My_Wallet_ETH["balance"]) # 특정 코인이나 원화의 잔고만 조회 # TypeError: must be real number, not dict
        output.write("# KRW-ETH 현재가 : %s 원\n" % price_KRW_ETH)
        # My_Wallet_ETH = upbit.get_balance(ticker="KRW-ETH") # <class 'float'>
        ETH_WON = float(My_Wallet_ETH["balance"]) * price_KRW_ETH
        print("# 현재 ETH원화 잔고 : %f 원, 수익율 : %f" % (ETH_WON, TEMP_ETH_Percent))

    if len(My_Wallet_KRW) != 0 :
        My_Wallet_KRW = My_Wallet_KRW["balance"]
        print("# 현재 원화 잔고 조회 : %s" % My_Wallet_KRW) # 특정 코인이나 원화의 잔고만 조회
        output.write("# 현재 원화 잔고 조회 : %s\n" % My_Wallet_KRW) # 특정 코인이나 원화의 잔고만 조회
        print("############################################################")
        output.write("############################################################\n")

    return len(res.json())

def Start_BIT_Trade(Call_Price_Possible, Call_Volume_Possible, File_Name) :

    OneDay_BUY_Count = 0 # 하루에 매수하는 회수를 제어하기 위한 Count, PUT(BUY) 호출할 때마다 1 Count (Start_BIT_Trade에서 최초 0부터 해서 시작)
    infinite_while = 1

    while infinite_while == 1 :
        output = open(File_Name, "a")

        # print("\n# Time Check : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        # output.write("\n# Time Check : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        now_time = datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")

        ############################################################
        # Loop 안에서 새롭게 조회를 해야 한다
        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
        }

        jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/accounts", headers=headers)

        if len(res.json()) < 2 :
            print("# 보유자산 1개 뿐임 (BIT or WON)")
            Temp_Check = res.json()[0]
            if Temp_Check['currency'] == 'KRW' :
                print("# 보유자산 1개 = WON만")
                continue
            elif Temp_Check['currency'] == 'BTC' :
                print("# 보유자산 1개 = BIT만)")
                continue
            elif Temp_Check['currency'] == 'ETH' :
                print("# 보유자산 1개 = BIT만)")
                continue
            else :
                print("# 보유자산 Error")
                continue

        My_Wallet_KRW, My_Wallet_BTC, My_Wallet_ETH = {}, {}, {}
        if len(res.json()) >= 2 :
            for x in res.json() :
                if x["currency"] == "KRW" :
                    My_Wallet_KRW = x
                elif x["currency"] == "BTC" :
                    My_Wallet_BIT = x
                elif x["currency"] == "ETH" :
                    My_Wallet_ETH = x
                else :
                    print("# ELSE")

        # print(My_Wallet_KRW) # {'currency': 'KRW', 'balance': '7087.59216207', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
        # print(My_Wallet_BIT) # {'currency': 'BTC', 'balance': '0.00013826', 'locked': '0.0', 'avg_buy_price': '73643511.93', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
        # print(My_Wallet_ETH) # {'currency': 'ETH', 'balance': '0.00198886', 'locked': '0.0', 'avg_buy_price': '2514000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}

        price_KRW_BTC = Current_BTC()
        price_KRW_ETH = Current_ETH()
        # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # <class 'float'>
        # price_KRW_ETH = pyupbit.get_current_price("KRW-ETH") # <class 'float'>
        # print(price_KRW_BTC) # <class 'float'>
        # print(My_Wallet_BIT['balance']) # <class 'str'>

        ############################################################

        # My_Wallet = upbit.get_balances(0)[0]
        My_Wallet_Money = float(My_Wallet_KRW['balance']) # 내 보유 WON

        # if not isinstance(My_Wallet_BIT, float) :
        if isinstance(My_Wallet_BIT, dict) :
            if len(My_Wallet_BIT) == 6 :
                My_BTC_WON = round(float(My_Wallet_BIT['balance']) * price_KRW_BTC, 1)
                TEMP_BTC_Current = price_KRW_BTC
                # print("# TEMP_BTC_Current : %s" % TEMP_BTC_Current)
                TEMP_BTC_Percent = ((TEMP_BTC_Current - float(My_Wallet_BIT['avg_buy_price'])) / float(My_Wallet_BIT['avg_buy_price'])) * 100
                TEMP_BTC_Percent = round(TEMP_BTC_Percent, 2)
                TEMP_BITWON = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
                # print("# BIT-Percent : %s" % TEMP_BTC_Percent)
                # print("# BITWON : %f" % TEMP_BITWON)
            else :
                print("# %s : BTC 없어서 매수" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : BTC 없어서 매수" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                __Make_Put("BUY", "BTC", 10000, 10000, OneDay_BUY_Count, File_Name)
                continue # 없어서 매수를 다시 했으므로 While문을 다시 시작
                # TEMP_BITWON = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>

        # if not isinstance(My_Wallet_ETH, float) :
        if isinstance(My_Wallet_ETH, dict) :
            if len(My_Wallet_ETH) == 6 :
                My_ETH_WON = round(float(My_Wallet_ETH['balance']) * price_KRW_ETH, 1)
                TEMP_ETH_Current = price_KRW_ETH
                # print("# TEMP_ETH_Current : %s" % TEMP_ETH_Current)
                TEMP_ETH_Percent = ((TEMP_ETH_Current - float(My_Wallet_ETH['avg_buy_price'])) / float(My_Wallet_ETH['avg_buy_price'])) * 100
                TEMP_ETH_Percent = round(TEMP_ETH_Percent, 2)
                TEMP_ETHWON = upbit.get_balance(ticker="KRW-ETH") # <class 'float'>
                # print("# ETH-Percent : %s" % TEMP_ETH_Percent)
                # print("# ETHWON : %f" % TEMP_ETHWON)
            else :
                print("# %s : ETH 없어서 매수" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : ETH 없어서 매수" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                __Make_Put("BUY", "ETH", 10000, 10000, OneDay_BUY_Count, File_Name)
                continue # 없어서 매수를 다시 했으므로 While문을 다시 시작
                # TEMP_ETHWON = upbit.get_balance(ticker="KRW-ETH") # <class 'float'>

        # if TEMP_BITWON < Min_Call_Price : # 현재 보유BIT가 10000원을 넘지 않을 경우 매수, Min_Call_Price = float(10000)
        if TEMP_BITWON < 0 : # 0보다 작다면 없는 것으로 판단하고, 소수점이라도 있으면 있다고 판단하여 없을 경우에 매수 호출
            # My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT ---> 자산 내 BIT수가 바뀌진 않지...
            print("# TEMP_BITWON")
            output.write("# TEMP_BITWON")
            __Make_Put("BUY", "BTC", 10000, 10000, OneDay_BUY_Count, File_Name)
            continue # 없어서 매수를 다시 했으므로 While문을 다시 시작

        if TEMP_ETHWON < 0 : # 0보다 작다면 없는 것으로 판단하고, 소수점이라도 있으면 있다고 판단하여 없을 경우에 매수 호출
            print("# TEMP_ETHWON")
            output.write("# TEMP_ETHWON")
            # My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT ---> 자산 내 BIT수가 바뀌진 않지...
            __Make_Put("BUY", "ETH", 10000, 10000, OneDay_BUY_Count, File_Name)
            continue # 없어서 매수를 다시 했으므로 While문을 다시 시작

        ############################################################
        # BTC Trading

        My_Wallet_BIT = upbit.get_balance("KRW-BTC") # KRW-XRP 조회
        # 수익실현하는 조건
        if TEMP_BTC_Percent > BTC_Upper_Value :
            print("# %s : 조건 OK ! (익절) ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %s, KRW-BIT가격: %s" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), BTC_Upper_Value, BTC_Lower_Value, TEMP_BTC_Percent, TEMP_BITWON, My_BTC_WON))
            output.write("# %s : 조건 OK ! (Upper) ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %s, KRW-BIT가격: %s\n" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), BTC_Upper_Value, BTC_Lower_Value, TEMP_BTC_Percent, TEMP_BITWON, My_BTC_WON))
            print("# %s : %s가 %s 보다 크다 ---> SELL!" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), TEMP_BTC_Percent, BTC_Upper_Value))
            output.write("# %s : %s가 %s 보다 크다 ---> SELL!\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), TEMP_BTC_Percent, BTC_Upper_Value))
            print("# %s : Make Put SELL : %s개 매도" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), My_Wallet_BIT)) # ---> 매도 때는 전체 bit 매도
            output.write("# %s : Make Put SELL : %s개 매도\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), My_Wallet_BIT))

            # 22:34:48 : Make Put SELL : {'currency': 'BTC', 'balance': '0.00007273', 'locked': '0.0', 'avg_buy_price': '687410000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}개 매도
            # testtest : {'currency': 'BTC', 'balance': '0.00007273', 'locked': '0.0', 'avg_buy_price': '687410000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}

            # Trade_Count = __Make_OrderLists("KRW-BTC") # ---> Return 값 : bid : 매수, ask : 매도
            My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT ---> 자산 내 BIT수가 바뀌진 않지...
            __Make_Put("SELL", "BTC", My_Wallet_BIT, Min_Call_Price, OneDay_BUY_Count, File_Name)
            # ---> Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10 계산값
            # ---> Call_Possible 값은 <--- My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT
            # ---> Min_Call_Price = float(10000)
            
        # 물타기하는 조건
        elif TEMP_BTC_Percent < BTC_Lower_Value :
            print("# %s : 조건 OK ! (물타기) ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %s, KRW-BIT가격: %s" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), BTC_Upper_Value, BTC_Lower_Value, TEMP_BTC_Percent, TEMP_BITWON, My_BTC_WON))
            output.write("# %s : 조건 OK ! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %s, KRW-BIT가격: %s\n" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), BTC_Upper_Value, BTC_Lower_Value, TEMP_BTC_Percent, TEMP_BITWON, My_BTC_WON))
            if My_Wallet_Money < Min_Call_Price : # My_Wallet_KRW = res.json()[0] ---> 내 보유 WON, Min_Call_Price = float(10000)
                print("# %s : 현재 잔액이 5,000원보다 적다" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : 현재 잔액이 5,000원보다 적다\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            else :
                print("# %s : BUY~!" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : BUY~!\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                print("# %s : Make Put BUY : %s원 매수" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Price_Possible))
                output.write("# %s : Make Put BUY : %s원 매수\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Price_Possible))

                Trade_Count = __Make_OrderLists("KRW-BTC") # ---> Return 값 : bid : 매수, ask : 매도

                if Trade_Count > 3 :
                # if "bid" in Trade_Count :
                    print("# %s : 매수 3건 이상 존재" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("# %s : 매수 3건 이상 존재\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                else :
                    print("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    __Make_Put("BUY", "BTC", Call_Price_Possible, Min_Call_Price, OneDay_BUY_Count, File_Name)
                    # ---> Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10 계산값
                    # ---> Call_Possible 값은 <--- My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT
                    # ---> Min_Call_Price = float(10000)
        else :
            # price_KRW_BTC = Current_BTC()
            # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC")
            print("#\n")
            output.write("#\n")
            print("# %s : 조건에 안들어온다 WAIT! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %f, KRW-BIT가격: %s" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), BTC_Upper_Value, BTC_Lower_Value, TEMP_BTC_Percent, TEMP_BITWON, My_BTC_WON))
            output.write("# %s : 조건에 안들어온다 WAIT! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %f, KRW-BIT가격: %s\n" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), BTC_Upper_Value, BTC_Lower_Value, TEMP_BTC_Percent, TEMP_BITWON, My_BTC_WON))

        ############################################################
        # ETH Trading

        My_Wallet_ETH = upbit.get_balance("KRW-ETH") # KRW-XRP 조회
        if TEMP_ETH_Percent > ETH_Upper_Value :
            print("# %s : 조건 OK ! (익절) ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 ETH는 %s, KRW-ETH가격: %s" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), ETH_Upper_Value, ETH_Lower_Value, TEMP_ETH_Percent, TEMP_ETHWON, My_ETH_WON))
            output.write("# %s : 조건 OK ! (Upper) ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 ETH는 %s, KRW-ETH가격: %s\n" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), ETH_Upper_Value, ETH_Lower_Value, TEMP_ETH_Percent, TEMP_ETHWON, My_ETH_WON))
            print("# %s : %s가 %s 보다 크다 ---> SELL!" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), TEMP_ETH_Percent, ETH_Upper_Value))
            output.write("# %s : %s가 %s 보다 크다 ---> SELL!\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), TEMP_ETH_Percent, ETH_Upper_Value))
            # print("# Make Put SELL : %s개 매도" % Call_Volume_Possible)
            # output.write("# Make Put SELL : %s개 매도\n" % Call_Volume_Possible)
            print("# %s : Make Put SELL : %s개 매도" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), My_Wallet_ETH)) # ---> 매도 때는 전체 bit 매도
            output.write("# %s : Make Put SELL : %s개 매도\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), My_Wallet_ETH))

            # 22:34:48 : Make Put SELL : {'currency': 'ETH', 'balance': '0.00007273', 'locked': '0.0', 'avg_buy_price': '687410000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}개 매도
            # testtest : {'currency': 'ETH', 'balance': '0.00007273', 'locked': '0.0', 'avg_buy_price': '687410000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}

            # Trade_Count = __Make_OrderLists("KRW-ETH") # ---> Return 값 : bid : 매수, ask : 매도
            My_Wallet_ETH = upbit.get_balance(ticker="KRW-ETH") # 자산 내 ETH ---> 자산 내 ETH수가 바뀌진 않지...
            __Make_Put("SELL", "ETH", My_Wallet_ETH, Min_Call_Price, OneDay_BUY_Count, File_Name)
            # ---> Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10 계산값
            # ---> Call_Possible 값은 <--- My_Wallet_ETH = upbit.get_balance(ticker="KRW-ETH") # 자산 내 ETH
            # ---> Min_Call_Price = float(10000)
            
        elif TEMP_ETH_Percent < ETH_Lower_Value :
            print("# %s : 조건 OK ! (물타기) ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 ETH는 %s, KRW-ETH가격: %s" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), ETH_Upper_Value, ETH_Lower_Value, TEMP_ETH_Percent, TEMP_ETHWON, My_ETH_WON))
            output.write("# %s : 조건 OK ! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 ETH는 %s, KRW-ETH가격: %s\n" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), ETH_Upper_Value, ETH_Lower_Value, TEMP_ETH_Percent, TEMP_ETHWON, My_ETH_WON))
            if My_Wallet_Money < Min_Call_Price : # My_Wallet_KRW = res.json()[0] ---> 내 보유 WON, Min_Call_Price = float(10000)
                print("# %s : 현재 잔액이 5,000원보다 적다" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : 현재 잔액이 5,000원보다 적다\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            else :
                print("# %s : BUY~!" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : BUY~!\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                print("# %s : Make Put BUY : %s원 매수" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Price_Possible))
                output.write("# %s : Make Put BUY : %s원 매수\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Price_Possible))

                Trade_Count = __Make_OrderLists("KRW-ETH") # ---> Return 값 : bid : 매수, ask : 매도

                if Trade_Count > 3 :
                # if "bid" in Trade_Count :
                    print("# %s : 매수 3건 이상 존재" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("# %s : 매수 3건 이상 존재\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                else :
                    print("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    __Make_Put("BUY", "ETH", Call_Price_Possible, Min_Call_Price, OneDay_BUY_Count, File_Name)
                    # ---> Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10 계산값
                    # ---> Call_Possible 값은 <--- My_Wallet_ETH = upbit.get_balance(ticker="KRW-ETH") # 자산 내 ETH
                    # ---> Min_Call_Price = float(10000)
        else :
            # price_KRW_ETH = Current_ETH()
            # price_KRW_ETH = pyupbit.get_current_price("KRW-ETH")
            print("# %s : 조건에 안들어온다 WAIT! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 ETH는 %f, KRW-ETH가격: %s" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), ETH_Upper_Value, ETH_Lower_Value, TEMP_ETH_Percent, TEMP_ETHWON, My_ETH_WON))
            output.write("# %s : 조건에 안들어온다 WAIT! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 ETH는 %f, KRW-ETH가격: %s\n" % \
            (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), ETH_Upper_Value, ETH_Lower_Value, TEMP_ETH_Percent, TEMP_ETHWON, My_ETH_WON))

        # ret = upbit.get_orders('KRW-XRP', 'wait', 5, 'asc')
        # ret = upbit.get_order('KRW-BTC', 'done')

        output.close()
        time.sleep(15)

if __name__ == '__main__' :

    # start_time = "09:00"
    # stop_time = "15:30"

    OneDay_BUY_Count = 0 # 하루에 매수하는 회수를 제어하기 위한 Count, PUT(BUY) 호출할 때마다 1 Count (Start_BIT_Trade에서 최초 0부터 해서 시작)

    DateTime = datetime.datetime.today().strftime("%Y%m%d")
    File_Name = "D:\\Python\\Log\\UPBit_History"+"_"+DateTime+".txt"
    output = open(File_Name, "a")
    output.write("# -*- coding: utf-8 -*-\n")

    # Current_Query(File_Name)
    Temp_My_Wallet = Current_Query(OneDay_BUY_Count, File_Name)

    ############################################################
    # Main Program
    Call_Price_Possible, Call_Volume_Possible = Call_Condition(File_Name)
    Start_BIT_Trade(Call_Price_Possible, Call_Volume_Possible, File_Name)
    ############################################################

    ############################################################
    # print(upbit.get_balance("KRW-BTC")) # KRW-XRP 조회
    # print(upbit.get_balance("KRW")) # 보유 현금 조회

    # print("############################################################")
    # test1 = __Make_OrderLists("KRW-BTC")
    # print("# TEST1")
    # print(test1)
    # test2 = __Make_OrderLists("KRW-ETH")
    # print("# TEST2")
    # print(test2)
    # if test2 > 3 :
    #     print("Over")

# BTC 마이너스만..
-0.12
-17.05
-5.8601626016260155
# ETH 마이너스만..
-0.04
-29.67
-7.727731092436977
# WHILE - TEMP_BTC_Current : 79510000.0
# WHILE - TEMP_ETH_Current : 3118000.0
# Current_Query Function Start
[{'currency': 'KRW', 'balance': '200959.76816152', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}, {'currency': 'XRP', 'balance': '9.07029478', 'locked': '0.0', 'avg_buy_price': '2205', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}, {'currency': 'ETH', 'balance': '0.00310848', 'locked': '0.0', 'avg_buy_price': '3217000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}]
<class 'list'>
# ELSE
# 20210416 16:22:09 : __Make_Put Function Start
# 20210416 16:22:09 : CALL Volume/Price : 10000.000000개 or 원
# 20210416 16:22:09 : Make call BUY
{'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 10000, 'ord_type': 'price'}
# Query : 
{'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 10000, 'ord_type': 'price'}
<cl

# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 16:25:30 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 79630000.0
# WHILE - TEMP_ETH_Current : 3121000.0
#

# 20210416 16:25:47 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.11, 내 BIT는 0.000251, KRW-BIT가격: 20021.4
# 20210416 16:25:48 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -2.98, 내 ETH는 0.00310848, KRW-ETH가격: 9701.6
# 20210416 16:25:48 : BUY~!
# 20210416 16:25:48 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 16:25:48 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 79647000.0
# WHILE - TEMP_ETH_Current : 3121000.0
#

# 20210416 16:26:06 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.13, 내 BIT는 0.000251, KRW-BIT가격: 20025.6
# 20210416 16:26:06 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -2.98, 내 ETH는 0.00310848, KRW-ETH가격: 9701.6
# 20210416 16:26:06 : BUY~!
# 20210416 16:26:06 : Make Put BUY 

# ELSE
# WHILE - TEMP_BTC_Current : 79550000.0
# WHILE - TEMP_ETH_Current : 3135000.0
#

# 20210416 16:30:44 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.01, 내 BIT는 0.000251, KRW-BIT가격: 20001.3
# 20210416 16:30:44 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -2.55, 내 ETH는 0.00310848, KRW-ETH가격: 9745.1
# 20210416 16:30:44 : BUY~!
# 20210416 16:30:44 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 16:30:45 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 79550000.0
# WHILE - TEMP_ETH_Current : 3135000.0
#

# 20210416 16:31:03 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.01, 내 BIT는 0.000251, KRW-BIT가격: 20001.3
# 20210416 16:31:03 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -2.55, 내 ETH는 0.00310848, KRW-ETH가격: 9745.1
# 20210416 16:31:03 : BUY~!
# 20210416 16:31:03 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 1

# WHILE - TEMP_ETH_Current : 3128000.0
#

# 20210416 16:35:41 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -0.27, 내 BIT는 0.000251, KRW-BIT가격: 19945.9
# 20210416 16:35:41 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -2.77, 내 ETH는 0.00310848, KRW-ETH가격: 9723.3
# 20210416 16:35:41 : BUY~!
# 20210416 16:35:41 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 16:35:41 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 79330000.0
# WHILE - TEMP_ETH_Current : 3126000.0
#

# 20210416 16:35:59 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -0.27, 내 BIT는 0.000251, KRW-BIT가격: 19945.9
# 20210416 16:35:59 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -2.83, 내 ETH는 0.00310848, KRW-ETH가격: 9717.1
# 20210416 16:35:59 : BUY~!
# 20210416 16:35:59 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 16:36:00 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_B

#

# 20210416 16:40:37 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -0.18, 내 BIT는 0.000251, KRW-BIT가격: 19964.3
# 20210416 16:40:37 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -2.67, 내 ETH는 0.00310848, KRW-ETH가격: 9732.7
# 20210416 16:40:37 : BUY~!
# 20210416 16:40:37 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 16:40:38 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 79334000.0
# WHILE - TEMP_ETH_Current : 3133000.0
#

# 20210416 16:40:56 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -0.26, 내 BIT는 0.000251, KRW-BIT가격: 19946.9
# 20210416 16:40:56 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -2.61, 내 ETH는 0.00310848, KRW-ETH가격: 9738.9
# 20210416 16:40:56 : BUY~!
# 20210416 16:40:56 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 16:40:56 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 79330000.0
# WHILE - TEMP_

...

#

# 20210416 21:44:36 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.08, 내 BIT는 0.000716, KRW-BIT가격: 56232.0
# 20210416 21:44:36 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -4.13, 내 ETH는 0.00310848, KRW-ETH가격: 9586.6
# 20210416 21:44:36 : BUY~!
# 20210416 21:44:36 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 21:44:37 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 78590000.0
# WHILE - TEMP_ETH_Current : 3083000.0
#

# 20210416 21:44:55 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.08, 내 BIT는 0.000716, KRW-BIT가격: 56232.7
# 20210416 21:44:55 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -4.17, 내 ETH는 0.00310848, KRW-ETH가격: 9583.4
# 20210416 21:44:55 : BUY~!
# 20210416 21:44:55 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 21:44:56 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 78556000.0
# WHILE - TEMP_ET

# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 21:49:35 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 78801000.0
# WHILE - TEMP_ETH_Current : 3094000.0
#

# 20210416 21:49:53 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.35, 내 BIT는 0.000716, KRW-BIT가격: 56383.7
# 20210416 21:49:53 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -3.82, 내 ETH는 0.00310848, KRW-ETH가격: 9617.6
# 20210416 21:49:53 : BUY~!
# 20210416 21:49:53 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 21:49:54 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 78730000.0
# WHILE - TEMP_ETH_Current : 3095000.0
#

# 20210416 21:50:11 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.26, 내 BIT는 0.000716, KRW-BIT가격: 56332.9
# 20210416 21:50:12 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -3.79, 내 ETH는 0.00310848, KRW-ETH가격: 9620.7
# 20210416 21:50:12 : BUY~!
# 20210416 21:50:12 : Make Put BUY 

# ELSE
# WHILE - TEMP_BTC_Current : 78651000.0
# WHILE - TEMP_ETH_Current : 3086000.0
#

# 20210416 21:54:55 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.16, 내 BIT는 0.000716, KRW-BIT가격: 56276.4
# 20210416 21:54:55 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -4.07, 내 ETH는 0.00310848, KRW-ETH가격: 9592.8
# 20210416 21:54:55 : BUY~!
# 20210416 21:54:55 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 21:54:56 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 78636000.0
# WHILE - TEMP_ETH_Current : 3084000.0
#

# 20210416 21:55:14 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.14, 내 BIT는 0.000716, KRW-BIT가격: 56265.6
# 20210416 21:55:14 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -4.13, 내 ETH는 0.00310848, KRW-ETH가격: 9586.6
# 20210416 21:55:14 : BUY~!
# 20210416 21:55:14 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 2

# WHILE - TEMP_ETH_Current : 3079000.0
#

# 20210416 21:59:57 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.01, 내 BIT는 0.000716, KRW-BIT가격: 56196.9
# 20210416 21:59:57 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -4.29, 내 ETH는 0.00310848, KRW-ETH가격: 9571.0
# 20210416 21:59:57 : BUY~!
# 20210416 21:59:57 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 21:59:57 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 78539000.0
# WHILE - TEMP_ETH_Current : 3079000.0
#

# 20210416 22:00:16 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.01, 내 BIT는 0.000716, KRW-BIT가격: 56196.2
# 20210416 22:00:16 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -4.29, 내 ETH는 0.00310848, KRW-ETH가격: 9571.0
# 20210416 22:00:16 : BUY~!
# 20210416 22:00:16 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 22:00:16 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC

#

# 20210416 22:04:57 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.22, 내 BIT는 0.000716, KRW-BIT가격: 56310.7
# 20210416 22:04:57 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -4.2, 내 ETH는 0.00310848, KRW-ETH가격: 9580.3
# 20210416 22:04:57 : BUY~!
# 20210416 22:04:57 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 22:04:58 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 78699000.0
# WHILE - TEMP_ETH_Current : 3086000.0
#

# 20210416 22:05:16 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 0.22, 내 BIT는 0.000716, KRW-BIT가격: 56310.7
# 20210416 22:05:16 : 조건 OK ! (물타기) ---> 수익 조건 : 1.5, 물타기 조건 : -1.5, 현재는 -4.07, 내 ETH는 0.00310848, KRW-ETH가격: 9592.8
# 20210416 22:05:16 : BUY~!
# 20210416 22:05:16 : Make Put BUY : 18095.002063769원 매수
# BID Count : 26
# ASK Count : 16
# Real BID Count #
# BID Count : 4
# 20210416 22:05:16 : 매수 3건 이상 존재
# ELSE
# WHILE - TEMP_BTC_Current : 78699000.0
# WHILE - TEMP_ETH